In [29]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import time
import statistics

import xgboost as xgb
import dask.array as da
import dask.distributed

In [30]:
client = Client('172.31.45.128:8786')
client.restart()
print(client)

<Client: 'tcp://172.31.45.128:8786' processes=0 threads=0, memory=0 B>


In [31]:
data = dd.read_csv("/home/ubuntu/data.csv", sample=2**20)
data = data.persist()

In [43]:
train, test = data.random_split([0.7, 0.3], random_state=42)

In [44]:
X_train = train.drop(labels=["tip_amount"], axis=1).copy()
Y_train = train["tip_amount"].copy()

In [47]:
X_test = test.drop(labels=["tip_amount"], axis=1).copy()
Y_test = test["tip_amount"].copy()

In [48]:
dtrain = xgb.dask.DaskDMatrix(client,X_train,Y_train)
output = xgb.dask.train(client
               ,{"verbosity": 2
                 , "tree_method": "hist"
                 , "objective": "reg:squarederror"}, dtrain
               , num_boost_round=4, evals=[(dtrain, "train")],
    )

In [53]:
prediction = xgb.dask.predict(client, output['booster'], X_test)

In [54]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")
print(f"R^2 Score: {r2_score_result}")

Mean Squared Error: 0.005478877054789918
Root Mean Squared Error: 0.07401943700670735
Mean Absolute Error: 0.04702530816817026
R^2 Score: 0.29599577982509606
